In [1]:
'''
匯入套件
'''
# 操作 browser 的 API
from selenium import webdriver

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 強制等待 (執行期間休息一下)
from time import sleep

# 整理 json 使用的工具
import json

# 執行 command 的時候用的
import os

# 子處理程序，用來取代 os.system 的功能
import subprocess

# 取得隨機數
import random

from urllib.parse import quote, unquote
from pprint import pprint
from time import sleep
from random import randint
import json
import pandas as pd

# 啟動瀏覽器工具的選項
my_options = webdriver.ChromeOptions()
my_options.add_argument("--start-maximized") #最大化視窗

# 使用 Chrome 的 WebDriver
driver = webdriver.Chrome(
    options = my_options,
)

# 建立儲存圖片、影片的資料夾
folderPath = 'taco-ith'
if not os.path.exists(folderPath):
    os.makedirs(folderPath)

# 放置爬取的資料
list_data = []

In [11]:
def visit():
    # 瀏覽器打開爬取頁面
    url = 'https://taco.ith.sinica.edu.tw/tdk/index.php?title=特殊:用戶登錄&returnto=熱蘭遮城日誌'
    driver.get(url)

def login():
    driver.find_element(By.CSS_SELECTOR, '#wpName1').send_keys('帳號')
    driver.find_element(By.CSS_SELECTOR, '#wpPassword1').send_keys('密碼')
    driver.find_element(By.CSS_SELECTOR, '#wpLoginAttempt').click()

def homepage():
    url = 'https://taco.ith.sinica.edu.tw/tdk/熱蘭遮城日誌'
    driver.get(url)

def get_main_links():
    a_elms = driver.find_elements(By.CSS_SELECTOR, 'div#mw-content-text > big > table[cellspacing][cellpadding] a[href][title]')
    for a in a_elms:
        list_data.append({
            'title': a.get_attribute('title'),
            'href': a.get_attribute('href'),
        })

def get_subpage_links():
    for i, d in enumerate(list_data):
        sleep(randint(1,4))
        driver.get(d['href'])
        a_elms = driver.find_elements(By.CSS_SELECTOR, 'div#mw-content-text big > ul > li > a[href][title]')
        list_data[i]['subpages'] = []
        for a in a_elms:
            list_data[i]['subpages'].append({
                'title': a.get_attribute('title'),
                'href': a.get_attribute('href'),
            })

    with open(f"{folderPath}/taco-ith.json", "w", encoding="utf8") as f:
        json.dump(list_data, f, ensure_ascii=False, indent=4)


def get_detail_links():
    with open(f"{folderPath}/taco-ith.json", "r", encoding="utf8") as f:
        list_data = json.load(f)

    for i, d in enumerate(list_data):
        for j, subpage in enumerate(d['subpages']):
            sleep(randint(1,4))
            driver.get(subpage['href'])
            a_elms = driver.find_elements(By.CSS_SELECTOR, 'div#mw-content-text big > ul > li > a[href][title]')
            list_data[i]['subpages'][j]['details'] = []
            for a in a_elms:
                list_data[i]['subpages'][j]['details'].append({
                    'title': a.get_attribute('title'),
                    'href': a.get_attribute('href'),
                })

    with open(f"{folderPath}/taco-ith.json", "w", encoding="utf8") as f:
        json.dump(list_data, f, ensure_ascii=False, indent=4)

def get_content():
    with open(f"{folderPath}/taco-ith.json", "r", encoding="utf8") as f:
        list_data = json.load(f)

    list_records = []
    count = 0
    columns = ['main_title', 'sub_title', 'detail_title', 'detail_href', 'content', 'references']

    for i, d in enumerate(list_data):
        for j, subpage in enumerate(d['subpages']):
            for k, detail in enumerate(subpage['details']):
                count += 1
                # sleep(0.2)
                driver.get(detail['href'])
                content = driver.find_element(By.CSS_SELECTOR, 'div#mw-content-text table[width] td[class=itharticle]').get_attribute('innerText')
                references = []
                if len(driver.find_elements(By.CSS_SELECTOR, 'div#mw-content-text > ol[class=references]')) > 0:
                    for span in driver.find_elements(By.CSS_SELECTOR, 'div#mw-content-text > ol[class=references] > li[id^=cite] > span[class=reference-text]'):
                        references.append(span.get_attribute('innerText'))
                list_records.append([
                    list_data[i]['title'],
                    list_data[i]['subpages'][j]['title'],
                    list_data[i]['subpages'][j]['details'][k]['title'],
                    list_data[i]['subpages'][j]['details'][k]['href'],
                    content,
                    '\n'.join(references)
                ])
            
        print("=" * 80)
        if not os.path.exists(f"{folderPath}/taco-ith.xlsx"):
            df = pd.DataFrame(list_records, columns=columns)
            df.to_excel(f"{folderPath}/taco-ith.xlsx", index=False)
            del df
            print('建立 taco-ith.xlsx')
        else:
            df = pd.read_excel(f"{folderPath}/taco-ith.xlsx")
            df2 = pd.DataFrame(list_records, columns=columns)
            df = pd.concat([df, df2], ignore_index=True)
            df.to_excel(f"{folderPath}/taco-ith.xlsx", index=False)
            del df, df2
            print('更新 taco-ith.xlsx')
        
        list_records.clear()

    print("=" * 80)
    print(count)

def close():
    driver.quit()


In [3]:
visit()

In [4]:
login()

In [10]:
homepage()

In [31]:
get_main_links()

In [32]:
get_subpage_links()

In [35]:
get_detail_links()

In [12]:
get_content()

建立 taco-ith.xlsx
已寫入 taco-ith.xlsx
更新 taco-ith.xlsx
已寫入 taco-ith.xlsx
更新 taco-ith.xlsx
已寫入 taco-ith.xlsx
更新 taco-ith.xlsx
已寫入 taco-ith.xlsx
更新 taco-ith.xlsx
已寫入 taco-ith.xlsx
更新 taco-ith.xlsx
已寫入 taco-ith.xlsx
更新 taco-ith.xlsx
已寫入 taco-ith.xlsx
更新 taco-ith.xlsx
已寫入 taco-ith.xlsx
更新 taco-ith.xlsx
已寫入 taco-ith.xlsx
更新 taco-ith.xlsx
已寫入 taco-ith.xlsx
更新 taco-ith.xlsx
已寫入 taco-ith.xlsx
更新 taco-ith.xlsx
已寫入 taco-ith.xlsx
更新 taco-ith.xlsx
已寫入 taco-ith.xlsx
更新 taco-ith.xlsx
已寫入 taco-ith.xlsx
更新 taco-ith.xlsx
已寫入 taco-ith.xlsx
更新 taco-ith.xlsx
已寫入 taco-ith.xlsx
更新 taco-ith.xlsx
已寫入 taco-ith.xlsx
更新 taco-ith.xlsx
已寫入 taco-ith.xlsx
更新 taco-ith.xlsx
已寫入 taco-ith.xlsx
更新 taco-ith.xlsx
已寫入 taco-ith.xlsx
更新 taco-ith.xlsx
已寫入 taco-ith.xlsx
更新 taco-ith.xlsx
已寫入 taco-ith.xlsx
更新 taco-ith.xlsx
已寫入 taco-ith.xlsx
更新 taco-ith.xlsx
已寫入 taco-ith.xlsx
更新 taco-ith.xlsx
已寫入 taco-ith.xlsx
更新 taco-ith.xlsx
已寫入 taco-ith.xlsx
更新 taco-ith.xlsx
已寫入 taco-ith.xlsx
更新 taco-ith.xlsx
已寫入 taco-ith.xlsx
更新 taco-ith.xlsx
已寫入

In [13]:
close()